In [2]:
from typing import Any, Type
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

# Initialize OpenAI's LLM
llm = ChatOpenAI(temperature=0.1)


# Define a function to add two numbers
def plus(a, b):
    return a + b


# Define the input schema for the calculator tool
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


# Define the calculator tool
class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = "Use this to perform sums of two numbers."
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    # Method to perform the actual calculation
    def _run(self, a, b):
        return a + b

    # Method to handle async calculations (if needed)
    async def _arun(self, a, b):
        raise NotImplementedError("This tool does not support async")


# Define the REACT prompt template
react_prompt_template = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
    template="""
    You are a helpful assistant. When given a task, you will use the tools at your disposal to accomplish it. Here are the tools you have:

    {tools}

    Tool Names: {tool_names}

    You should take the input, decide which tool to use, and provide the output.

    Input: {input}

    {agent_scratchpad}
    """,
)

# Initialize tools
tools = [CalculatorTool()]

# Create the agent using LLM, tools, and prompt template
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt_template,
)

# Set up the agent executor with increased iteration/time limits
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
    max_iterations=10,
    max_execution_time=120,
)

# Test the CalculatorTool directly
calculator = CalculatorTool()
result = calculator._run(a=10, b=20)
print(f"Calculator result: {result}")  # Should print: Calculator result: 30

# Define a simpler prompt
simple_prompt = "Cost of $10 + $20"

# Execute the simpler prompt using the agent executor
response = agent_executor.invoke(
    {
        "input": simple_prompt,
        "tools": tools,
        "tool_names": "CalculatorTool: Use this to perform sums of two numbers.",
        "agent_scratchpad": "",
    }
)
print(response)

# Define the original complex prompt
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

# Execute the original prompt using the agent executor
response = agent_executor.invoke(
    {
        "input": prompt,
        "tools": tools,
        "tool_names": "CalculatorTool: Use this to perform sums of two numbers.",
        "agent_scratchpad": "",
    }
)
print(response)

Calculator result: 30
{'input': 'Cost of $10 + $20', 'tools': [CalculatorTool()], 'tool_names': 'CalculatorTool: Use this to perform sums of two numbers.', 'agent_scratchpad': '', 'output': 'Agent stopped due to iteration limit or time limit.'}
{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12', 'tools': [CalculatorTool()], 'tool_names': 'CalculatorTool: Use this to perform sums of two numbers.', 'agent_scratchpad': '', 'output': 'Agent stopped due to iteration limit or time limit.'}
